In [ ]:
# ============================================================================
# RDD2022 MODEL EVALUATION - GOOGLE COLAB (FIXED)
# ============================================================================

print("="*80)
print("🚀 RDD2022 MODEL EVALUATION IN GOOGLE COLAB")
print("="*80)

# ============================================================================
# 1. MOUNT GOOGLE DRIVE
# ============================================================================

print("\n📁 Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("✅ Drive mounted!")

# ============================================================================
# 2. INSTALL ULTRALYTICS
# ============================================================================

print("\n📦 Installing Ultralytics...")
!pip install -q ultralytics

from ultralytics import YOLO
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from tqdm import tqdm
import zipfile
import warnings
warnings.filterwarnings('ignore')

print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

# ============================================================================
# 3. EXTRACT & FIND DATASET
# ============================================================================

print("\n📦 Preparing dataset...")

DATASET_ZIP = '/content/drive/MyDrive/RDD.zip'

# Extract if not already done
if not Path('/content/RDD2022').exists():
    print(f"   Extracting {DATASET_ZIP}...")
    print("   This takes 5-10 minutes...")
    with zipfile.ZipFile(DATASET_ZIP, 'r') as zip_ref:
        zip_ref.extractall('/content/')
    print("✅ Extracted!")

# Find the correct dataset path
print("\n🔍 Finding dataset structure...")
!ls -la /content/

# Check possible paths
possible_roots = [
    Path('/content/RDD2022'),
    Path('/content/RDD'),
    Path('/content/RDD_SPLIT'),
]

DATASET_ROOT = None
for root in possible_roots:
    if root.exists():
        print(f"   Found: {root}")
        # List what's inside
        subdirs = list(root.iterdir())
        print(f"   Contents: {[d.name for d in subdirs[:5]]}")

        # Check for RDD_SPLIT subdirectory
        if (root / 'RDD_SPLIT').exists():
            DATASET_ROOT = root / 'RDD_SPLIT'
            break
        # Check if root itself has test/train/val
        elif (root / 'test').exists():
            DATASET_ROOT = root
            break

if DATASET_ROOT is None:
    print("❌ Cannot find dataset! Checking /content/ structure:")
    !find /content -name "test" -type d 2>/dev/null | head -20
    raise FileNotFoundError("Dataset not found!")

print(f"\n✅ Dataset root: {DATASET_ROOT}")

# Verify test set
TEST_PATH = DATASET_ROOT / 'test'
if TEST_PATH.exists():
    test_images = list((TEST_PATH / 'images').glob('*.jpg'))
    test_labels = list((TEST_PATH / 'labels').glob('*.txt'))
    print(f"✅ Test images: {len(test_images)}")
    print(f"✅ Test labels: {len(test_labels)}")
else:
    print(f"❌ Test path not found: {TEST_PATH}")
    print("   Available directories:")
    for d in DATASET_ROOT.iterdir():
        print(f"     - {d.name}")
    raise FileNotFoundError("Test directory not found!")

# ============================================================================
# 4. CONFIGURE PATHS
# ============================================================================

print("\n📂 Configuring paths...")

MODEL_PATHS = {
    'YOLOv8n @ 640': '/content/drive/MyDrive/best_model/yolov8n_640_best.pt',
    'YOLOv8s @ 640': '/content/drive/MyDrive/best_model/yolov8s_640_best.pt',
    'YOLOv8s @ 1024': '/content/drive/MyDrive/best_model/yolov8s_1024_best.pt',
}

OUTPUT_DIR = Path('/content/drive/MyDrive/RDD2022_Evaluation_Results')
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

CLASS_NAMES = [
    'Longitudinal crack',
    'Transverse crack',
    'Alligator crack',
    'Other corruption',
    'Pothole'
]

# Verify models
print("\n🔍 Checking models...")
for name, path in MODEL_PATHS.items():
    if Path(path).exists():
        size = Path(path).stat().st_size / (1024*1024)
        print(f"  ✅ {name}: {size:.1f} MB")

# ============================================================================
# 5. CREATE DATA.YAML WITH CORRECT PATHS
# ============================================================================

print("\n📝 Creating data.yaml...")

# Create data.yaml with VERIFIED paths
data_yaml_content = f"""path: {DATASET_ROOT}
train: train/images
val: test/images
test: test/images

nc: 5
names:
  - Longitudinal crack
  - Transverse crack
  - Alligator crack
  - Other corruption
  - Pothole
"""

data_yaml_path = '/content/data.yaml'
with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"✅ Created: {data_yaml_path}")
print(f"   Dataset path: {DATASET_ROOT}")
print(f"   Test images: {len(test_images)}")

# ============================================================================
# 6. EVALUATION FUNCTION
# ============================================================================

def evaluate_model(model_path, model_name):
    """Evaluate a single model"""

    print("\n" + "="*80)
    print(f"🔍 EVALUATING: {model_name}")
    print("="*80)

    if not Path(model_path).exists():
        print(f"❌ Model not found")
        return None

    try:
        print("\n📥 Loading model...")
        model = YOLO(model_path)

        imgsz = 1024 if '1024' in model_name else 640
        print(f"✅ Image size: {imgsz}")

        print(f"\n⏳ Running validation (15-30 minutes)...")

        results = model.val(
            data=data_yaml_path,
            split='test',
            batch=16,
            imgsz=imgsz,
            conf=0.001,
            iou=0.6,
            device=0,
            verbose=True
        )

        # Extract metrics
        metrics = {
            'Model': model_name,
            'Image_Size': imgsz,
            'Parameters': f"{sum(p.numel() for p in model.model.parameters())/1e6:.1f}M",
            'mAP@50': float(results.box.map50),
            'mAP@50-95': float(results.box.map),
            'Precision': float(results.box.mp),
            'Recall': float(results.box.mr),
            'F1-Score': float(2 * (results.box.mp * results.box.mr) /
                            (results.box.mp + results.box.mr + 1e-6))
        }

        # Per-class metrics
        per_class_metrics = []
        for i, class_name in enumerate(CLASS_NAMES):
            per_class_metrics.append({
                'Model': model_name,
                'Class': class_name,
                'Precision': float(results.box.p[i]),
                'Recall': float(results.box.r[i]),
                'mAP@50': float(results.box.ap50[i]),
                'mAP@50-95': float(results.box.ap[i]),
                'F1-Score': float(2 * (results.box.p[i] * results.box.r[i]) /
                                 (results.box.p[i] + results.box.r[i] + 1e-6))
            })

        # Print results
        print(f"\n{'='*70}")
        print(f"📊 RESULTS: {model_name}")
        print(f"{'='*70}")
        print(f"mAP@50:    {metrics['mAP@50']:.4f} ({metrics['mAP@50']*100:.2f}%)")
        print(f"mAP@50-95: {metrics['mAP@50-95']:.4f}")
        print(f"Precision: {metrics['Precision']:.4f}")
        print(f"Recall:    {metrics['Recall']:.4f}")
        print(f"F1-Score:  {metrics['F1-Score']:.4f}")

        print(f"\n📊 TOP 3 CLASSES:")
        sorted_classes = sorted(per_class_metrics, key=lambda x: x['mAP@50'], reverse=True)
        for pc in sorted_classes[:3]:
            print(f"  {pc['Class']}: {pc['mAP@50']:.4f}")

        print(f"\n✅ Complete!")

        return {
            'overall': metrics,
            'per_class': per_class_metrics
        }

    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# ============================================================================
# 7. EVALUATE ALL MODELS
# ============================================================================

print("\n" + "="*80)
print("🚀 STARTING EVALUATION")
print("="*80)

all_results = {}

for model_name, model_path in MODEL_PATHS.items():
    result = evaluate_model(model_path, model_name)
    if result:
        all_results[model_name] = result
        print(f"\n✅ {model_name} done! ({len(all_results)}/3)")

# ============================================================================
# 8. CREATE RESULTS
# ============================================================================

if all_results:
    print("\n" + "="*80)
    print("📊 CREATING RESULTS")
    print("="*80)

    # Overall table
    overall_data = [r['overall'] for r in all_results.values()]
    comparison_df = pd.DataFrame(overall_data)

    print("\n📊 FINAL COMPARISON:")
    print(comparison_df.to_string(index=False))

    # Save
    comparison_df.to_csv(OUTPUT_DIR / 'overall_comparison.csv', index=False)
    print(f"\n✅ Saved to: {OUTPUT_DIR}")

    # Per-class
    all_per_class = []
    for r in all_results.values():
        all_per_class.extend(r['per_class'])
    per_class_df = pd.DataFrame(all_per_class)
    per_class_df.to_csv(OUTPUT_DIR / 'per_class_comparison.csv', index=False)

    # Pivot tables
    for metric in ['mAP@50', 'Precision', 'Recall', 'F1-Score']:
        pivot = per_class_df.pivot(index='Class', columns='Model', values=metric)
        print(f"\n📊 {metric}:")
        print(pivot.to_string())
        pivot.to_csv(OUTPUT_DIR / f'{metric.lower().replace("@","_").replace("-","_")}.csv')

    # Chart
    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(comparison_df))
    width = 0.2
    metrics = ['mAP@50', 'Precision', 'Recall', 'F1-Score']

    for i, m in enumerate(metrics):
        ax.bar(x + width*i - width*1.5, comparison_df[m], width, label=m)

    ax.set_xlabel('Model')
    ax.set_ylabel('Score')
    ax.set_title('Performance Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(comparison_df['Model'], rotation=15, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim([0, 1])
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'comparison.png', dpi=300)
    plt.close()

    # Best model
    best_idx = comparison_df['mAP@50'].idxmax()
    best = comparison_df.loc[best_idx]
    print(f"\n🏆 BEST: {best['Model']} - {best['mAP@50']:.4f}")

    print(f"\n📁 All results saved to:")
    print(f"   {OUTPUT_DIR}")

print("\n🎉 DONE!")

🚀 RDD2022 MODEL EVALUATION IN GOOGLE COLAB

📁 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive mounted!

📦 Installing Ultralytics...
✅ PyTorch: 2.9.0+cu126
✅ CUDA: True
✅ GPU: Tesla T4

📦 Preparing dataset...
   Extracting /content/drive/MyDrive/RDD.zip...
   This takes 5-10 minutes...
✅ Extracted!

🔍 Finding dataset structure...
total 32
drwxr-xr-x 1 root root 4096 Dec 16 10:34 .
drwxr-xr-x 1 root root 4096 Dec 16 10:28 ..
drwxr-xr-x 4 root root 4096 Dec  9 14:41 .config
-rw-r--r-- 1 root root  198 Dec 16 10:34 data.yaml
drwx------ 5 root root 4096 Dec 16 10:29 drive
drwxr-xr-x 5 root root 4096 Dec 16 10:33 RDD_SPLIT
drwxr-xr-x 3 root root 4096 Dec 16 10:34 runs
drwxr-xr-x 1 root root 4096 Dec  9 14:42 sample_data
   Found: /content/RDD_SPLIT
   Contents: ['test', 'val', 'train']

✅ Dataset root: /content/RDD_SPLIT
✅ Test images: 5758
✅ Test labels: 5758

📂 Configuring path